In [1]:
import keras 
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Flatten
from keras import optimizers
from keras.models import Sequential
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

2024-03-01 12:07:13.583149: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-01 12:07:13.627170: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 12:07:13.938276: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 12:07:13.939399: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 12:07:15.057848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                   horizontal_flip = True)


training_set = train_datagen.flow_from_directory(r'/home/hazeeba/Documents/Luminar_internship/Parkinson/P5_Parkinson/park_img/spiral/train',
                                                 target_size = (224, 224),
                                                 batch_size = 8,
                                                 subset="training",
                                                 class_mode = 'categorical')

validation_set = train_datagen.flow_from_directory(r'/home/hazeeba/Documents/Luminar_internship/Parkinson/P5_Parkinson/park_img/spiral/test',
                                                 target_size = (224, 224),
                                                 batch_size = 8,
                                                subset="validation",
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(r'/home/hazeeba/Documents/Luminar_internship/Parkinson/P5_Parkinson/park_img/spiral/test',
                                            target_size = (224, 224),
                                            batch_size = 8,
                                            class_mode = 'categorical')

#color_mode = "grayscale"


STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
STEP_SIZE_VALID=validation_set.n//validation_set.batch_size
STEP_SIZE_TEST=test_set.n//test_set.batch_size

Found 288 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Found 180 images belonging to 2 classes.


In [3]:
training_set


In [4]:
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#here [3] denotes for RGB images(3 channels)

#don't train existing weights
for layer in vgg.layers:
 layer.trainable = False
 
x = Flatten()(vgg.output)
prediction = Dense(2, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.compile(loss='binary_crossentropy',
                    optimizer=optimizers.Adam(),
                    metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=1, save_best_only=True)
callbacks = [checkpoint, lr_reducer]
start = datetime.now()
# history = model.fit_generator(training_set, 
#                     steps_per_epoch=STEP_SIZE_TRAIN, 
#                     epochs = 18, verbose=5, 
#                     validation_data = validation_set, 
#                     validation_steps = STEP_SIZE_VALID)
# duration = datetime.now() - start
# print("Training completed in time: ", duration)

# Assuming you have a `training_set` generator
# and `batch_size` is the batch size used in the generator

batch_size=8
total_samples = len(training_set) * batch_size
steps_per_epoch = total_samples // batch_size

# Make sure steps_per_epoch is at least 1
steps_per_epoch = max(steps_per_epoch, 1)

history = model.fit_generator(
    training_set,
    steps_per_epoch=steps_per_epoch,
    epochs=18,
    verbose=5,
    validation_data=validation_set,
    validation_steps=len(validation_set),  # or calculate based on validation data size
    callbacks=[checkpoint, lr_reducer]
)
duration = datetime.now() - start
print("Training completed in time: ", duration)


/tmp/ipykernel_8123/1185727071.py:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/18

Epoch 1: val_loss improved from inf to 0.91521, saving model to mymodel.h5


/home/hazeeba/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/18

Epoch 2: val_loss did not improve from 0.91521
Epoch 3/18

Epoch 3: val_loss did not improve from 0.91521
Epoch 4/18

Epoch 4: val_loss did not improve from 0.91521
Epoch 5/18

Epoch 5: val_loss did not improve from 0.91521
Epoch 6/18

Epoch 6: val_loss did not improve from 0.91521
Epoch 7/18

Epoch 7: val_loss did not improve from 0.91521
Epoch 8/18

Epoch 8: val_loss did not improve from 0.91521
Epoch 9/18

Epoch 9: val_loss did not improve from 0.91521
Epoch 10/18

Epoch 10: val_loss did not improve from 0.91521
Epoch 11/18

Epoch 11: val_loss did not improve from 0.91521
Epoch 12/18

Epoch 12: val_loss did not improve from 0.91521
Epoch 13/18

Epoch 13: val_loss did not improve from 0.91521
Epoch 14/18

Epoch 14: val_loss did not improve from 0.91521
Epoch 15/18

Epoch 15: val_loss did not improve from 0.91521
Epoch 16/18

Epoch 16: val_loss did not improve from 0.91521
Epoch 17/18

Epoch 17: val_loss did not improve from 0.91521
Epoch 18/18

Epoch 18: val_loss did not 

In [6]:
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

23/23 [==============================] - 17s 748ms/step - loss: 0.4102 - accuracy: 0.8778
Test Loss: 0.41019129753112793
Test accuracy: 0.8777777552604675


In [7]:
# Image Data Generators for the "wave" dataset
train_datagen_wave = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    validation_split=0.2,
    horizontal_flip = True
)

test_datagen_wave = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    validation_split=0.2,
    horizontal_flip = True
)

# Directories for the "wave" dataset
train_dir_wave = r'/home/hazeeba/Documents/Luminar_internship/Parkinson/P5_Parkinson/park_img/wave/train'
test_dir_wave = r'/home/hazeeba/Documents/Luminar_internship/Parkinson/P5_Parkinson/park_img/wave/test'

# Flow from Directory for the "wave" dataset
training_set_wave = train_datagen_wave.flow_from_directory(
    train_dir_wave,
    target_size = (224, 224),
    batch_size = 8,
    subset="training",
    class_mode = 'categorical'
)

validation_set_wave = train_datagen_wave.flow_from_directory(
    train_dir_wave,
    target_size = (224, 224),
    batch_size = 8,
    subset="validation",
    class_mode = 'categorical'
)

test_set_wave = test_datagen_wave.flow_from_directory(
    test_dir_wave,
    target_size = (224, 224),
    batch_size = 8,
    class_mode = 'categorical'
)

Found 288 images belonging to 2 classes.
Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [8]:
# Model Architecture
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(2, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

# Callbacks
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                cooldown=0,
                                patience=5,
                                min_lr=0.5e-6)

checkpoint = ModelCheckpoint(filepath='mymodel_wave.h5', 
                              verbose=1, 
                              save_best_only=True)

callbacks = [checkpoint, lr_reducer]

# Training
batch_size = 8
total_samples = len(training_set_wave) * batch_size
steps_per_epoch = total_samples // batch_size
steps_per_epoch = max(steps_per_epoch, 1)

start = datetime.now()

history = model.fit_generator(
    training_set_wave,
    steps_per_epoch=steps_per_epoch,
    epochs=18,
    verbose=5,
    validation_data=validation_set_wave,
    validation_steps=len(validation_set_wave),
    callbacks=[checkpoint, lr_reducer]
)

duration = datetime.now() - start
print("Training completed in time: ", duration)



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

/tmp/ipykernel_8123/1958012368.py:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(



Epoch 1: val_loss improved from inf to 0.98228, saving model to mymodel_wave.h5
Epoch 2/18

Epoch 2: val_loss improved from 0.98228 to 0.45682, saving model to mymodel_wave.h5
Epoch 3/18

Epoch 3: val_loss did not improve from 0.45682
Epoch 4/18

Epoch 4: val_loss did not improve from 0.45682
Epoch 5/18

Epoch 5: val_loss improved from 0.45682 to 0.36792, saving model to mymodel_wave.h5
Epoch 6/18

Epoch 6: val_loss did not improve from 0.36792
Epoch 7/18

Epoch 7: val_loss improved from 0.36792 to 0.31845, saving model to mymodel_wave.h5
Epoch 8/18

Epoch 8: val_loss did not improve from 0.31845
Epoch 9/18

Epoch 9: val_loss did not improve from 0.31845
Epoch 10/18

Epoch 10: val_loss did not improve from 0.31845
Epoch 11/18

Epoch 11: val_loss did not improve from 0.31845
Epoch 12/18

Epoch 12: val_loss did not improve from 0.31845
Epoch 13/18

Epoch 13: val_loss did not improve from 0.31845
Epoch 14/18

Epoch 14: val_loss did not improve from 0.31845
Epoch 15/18

Epoch 15: val_loss

In [9]:
# Evaluation
score = model.evaluate(test_set_wave)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

4/4 [==============================] - 3s 636ms/step - loss: 0.3527 - accuracy: 0.8000
Test Loss: 0.3527430295944214
Test accuracy: 0.800000011920929


In [26]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# Load the trained models
model_spiral = load_model('mymodel.h5')  # Trained model for spiral dataset
model_wave = load_model('mymodel_wave.h5')  # Trained model for wave dataset

# Path to the new image
new_image_path = '/home/hazeeba/Documents/Luminar_internship/Parkinson/Prediction_images/pw4.png'  # Replace with the path to your new image

# Load and preprocess the new image
new_img = image.load_img(new_image_path, target_size=(224, 224))
new_img_array = image.img_to_array(new_img)
new_img_array = np.expand_dims(new_img_array, axis=0)
new_img_array /= 255.0  # Normalize the image

# Make predictions using both models
prediction_spiral = model_spiral.predict(new_img_array)
prediction_wave = model_wave.predict(new_img_array)

# Combine predictions from both models
combined_prediction = (prediction_spiral + prediction_wave) / 2  # Take the average

# Convert the combined predicted probabilities to class labels
predicted_class = np.argmax(combined_prediction)

# Define the class labels
class_labels = ['Healthy', 'Parkinson']

# Get the label of the image
predicted_label = class_labels[predicted_class]

# Display the predicted label
print("Predicted Label:", predicted_label)


1/1 [==============================] - 0s 222ms/step
Predicted Label: Parkinson
